In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset, TensorDataset
from torch.optim import SGD, Adam
device = 'cuda' if torch.cuda.is_available() else 'cpu'

from torchvision import datasets
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
x_train = torch.tensor([
    [
        [
            [1,2,3,4],[2,3,4,5],[5,6,7,8],[1,3,4,5]
        ]
    ],[
        [
            [-1,2,3,-4],[2,-3,4,5],[-5,6,-7,8],[-1,-3,-4,-5]
        ]
    ]
]).to(device).float()
# x_train => 2x1x4x4 | NxCxHxW
# N - Number of images (batch size)
# C - Number of channels
# H - Height of the image
# W - Width of the image
x_train /= 8
y_train = torch.tensor([[0],[1]]).to(device).float()

In [3]:
def get_model():
    model = nn.Sequential(
        nn.Conv2d(1, 1, kernel_size=3),
        nn.MaxPool2d(2),
        nn.ReLU(),
        nn.Flatten(),
        nn.Linear(1,1),
        nn.Sigmoid()
    ).to(device)
    loss_fn = nn.BCELoss()
    optimizer = Adam(model.parameters(), lr=1e-2)
    return model, loss_fn, optimizer

In [4]:
!pip install torch_summary
from torchsummary import summary
model, loss_fn, optimizer = get_model()
summary(model, x_train)

Layer (type:depth-idx)                   Output Shape              Param #
├─Conv2d: 1-1                            [-1, 1, 2, 2]             10
├─MaxPool2d: 1-2                         [-1, 1, 1, 1]             --
├─ReLU: 1-3                              [-1, 1, 1, 1]             --
├─Flatten: 1-4                           [-1, 1]                   --
├─Linear: 1-5                            [-1, 1]                   2
├─Sigmoid: 1-6                           [-1, 1]                   --
Total params: 12
Trainable params: 12
Non-trainable params: 0
Total mult-adds (M): 0.00
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00


Layer (type:depth-idx)                   Output Shape              Param #
├─Conv2d: 1-1                            [-1, 1, 2, 2]             10
├─MaxPool2d: 1-2                         [-1, 1, 1, 1]             --
├─ReLU: 1-3                              [-1, 1, 1, 1]             --
├─Flatten: 1-4                           [-1, 1]                   --
├─Linear: 1-5                            [-1, 1]                   2
├─Sigmoid: 1-6                           [-1, 1]                   --
Total params: 12
Trainable params: 12
Non-trainable params: 0
Total mult-adds (M): 0.00
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00

In [5]:
def train_batch(x, y, model, loss_fn, optimizer):
    model.train()
    prediction = model(x)
    batch_loss = loss_fn(prediction, y)
    batch_loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    return batch_loss.item()

In [6]:
trn_dl = DataLoader(TensorDataset(x_train, y_train))

In [7]:
for epoch in range(2000):
    for ix, batch in enumerate(iter(trn_dl)):
        # print(ix)
        x, y = batch
        batch_loss = train_batch(x, y, model, loss_fn, optimizer)

In [8]:
model(x_train[:1])
# x_train[:1]

tensor([[0.5001]], device='cuda:0', grad_fn=<SigmoidBackward0>)

In [9]:
list(model.children())

[Conv2d(1, 1, kernel_size=(3, 3), stride=(1, 1)),
 MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),
 ReLU(),
 Flatten(start_dim=1, end_dim=-1),
 Linear(in_features=1, out_features=1, bias=True),
 Sigmoid()]

In [10]:
(cnn_w, cnn_b), (lin_w, lin_b) = [(layer.weight.data, layer.bias.data) for layer in list(model.children()) if hasattr(layer, 'weight')]

In [11]:
print(cnn_w, cnn_b)
print(lin_w, lin_b)

tensor([[[[ 0.2236, -0.2624,  0.1156],
          [ 0.2373,  0.0339, -0.0294],
          [-0.1324, -0.2743,  0.0959]]]], device='cuda:0') tensor([-0.2732], device='cuda:0')
tensor([[-0.0637]], device='cuda:0') tensor([0.0003], device='cuda:0')


In [12]:
h_im, w_im = x_train.shape[2:]
h_conv, w_conv = cnn_w.shape[2:]
sumprod = torch.zeros((h_im - h_conv + 1, w_im - w_conv + 1))

In [14]:
for i in range(h_im - h_conv + 1):
    for j in range(w_im - w_conv + 1):
        img_subset = x_train[0, 0, i:(i+3), j:(j+3)]
        model_filter = cnn_w.reshape(3,3)
        val = torch.sum(img_subset*model_filter) + cnn_b
        sumprod[i,j] = val

In [15]:
sumprod.clamp_min_(0)

tensor([[0., 0.],
        [0., 0.]])

In [16]:
pooling_layer_output = torch.max(sumprod)

In [17]:
intermediate_output_value = pooling_layer_output * lin_w + lin_b

In [18]:
from torch.nn import functional as F # torch library for numpy like functions
print(F.sigmoid(intermediate_output_value))

tensor([[0.5001]], device='cuda:0')
